In [1]:
!pip install dash 
!pip install more_itertools
!pip install pandas
!pip install plotly

In [2]:
import dash
import more_itertools
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

In [3]:
data = pd.read_csv('./historical_automobile_sales.csv')
data.head()

,Date,Year,Month,Recession,Consumer_Confidence,Seasonality_Weight,Price,Advertising_Expenditure,Competition,GDP,Growth_Rate,unemployment_rate,Automobile_Sales,Vehicle_Type,City
0,1/31/1980,1980,Jan,1,108.24,0.50,27483.571,1558,7,60.223,0.010000,5.4,456.0,Supperminicar,Georgia
1,2/29/1980,1980,Feb,1,98.75,0.75,24308.678,3048,4,45.986,-0.309594,4.8,555.9,Supperminicar,New York
2,3/31/1980,1980,Mar,1,107.48,0.20,28238.443,3137,3,35.141,-0.308614,3.4,620.0,Mediumfamilycar,New York
3,4/30/1980,1980,Apr,1,115.01,1.00,32615.149,1653,7,45.673,0.230596,4.2,702.8,Supperminicar,Illinois
4,5/31/1980,1980,May,1,98.72,0.20,23829.233,1319,4,52.997,0.138197,5.3,770.4,Smallfamiliycar,California


In [4]:
# Initialize the Dash app
app = dash.Dash(__name__)
app.config.suppress_callback_exceptions = True
# Set the title of the dashboard
app.title = "Automobile Statistics Dashboard"
#---------------------------------------------------------------------------------
# Create the dropdown menu options

dropdown_options = [
    {'label': 'Yearly Sales', 'value': data['Year']},
    {'label': 'Recession Period Statistics', 'value': data['Month']}
]
# List of years 
year_list = [i for i in range(1980, 2024, 1)]

#---------------------------------------------------------------------------------------

In [6]:

app.layout = html.Div([
    html.H1("Automobile Statistics Dashboard",
            style={'textAlign': 'center', 'color': 'red'}),
    html.Div([
        html.Div(
            html.Label("Select Statistics:",
                   style={ 'color': 'white','font-weight': 'bold','font': '18px Arial, Helvetica, sans-serif'}
                   ),
            style={'width': '60%','height':'30px','margin-left':'50px','padding':'15px'}

        ),

        dcc.Dropdown(
            id='select-statistics',
            options=[
                {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
            ],
            value='Yearly Statistics',
            placeholder='Select Statistics',
            style={
    'font-weight': 'bold',
    'font': '18px Arial, Helvetica, sans-serif'}
        )
    ]),
    html.Div(dcc.Dropdown(
            id='select-year',
            options=[{'label': i, 'value': i} for i in year_list],
            value=1980
        )),
    html.Div([
        html.Div(id='output-container', className='output-container', style={'width': '100%','margin':'auto', 'display': 'inline-block'}),
    ])
])

In [7]:
@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='select-statistics', component_property='value'))
def update_input_container(selected_statistics):
    if selected_statistics == 'Yearly Statistics':
        return False
    else:
        return True
    
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='select-statistics', component_property='value'),
    Input(component_id='select-year', component_property='value')])

def update_output_container(selected_statistics, input_year):
    if selected_statistics == 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = data[data['Recession'] == 1]
        yearly_rec=recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        # Plotting the line graph
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, 
                x='Year',
                y='Automobile_Sales',
                title="Automobile sales fluctuate over Recession Period"))
        
        average_sales = recession_data.groupby("Vehicle_Type")['Automobile_Sales'].mean().reset_index()                 
        R_chart2  = dcc.Graph(
            figure=px.bar(average_sales,
            x='Vehicle_Type',
            y='Automobile_Sales',
            title="Average number of vehicle sold "))
        
        exp_rec= recession_data.groupby(['Vehicle_Type'])['Advertising_Expenditure'].count().reset_index()

        R_chart3 = dcc.Graph(
                    figure=px.pie(exp_rec,
                    values='Advertising_Expenditure',
                 names='Vehicle_Type',
                 title="Expenditure share by vehicle type during recessions"
                )
        )
        unemp_data= recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(figure=px.bar(unemp_data,
        x='Vehicle_Type',
        y='Automobile_Sales',
        color='#f2defe',
        labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Automobile Sales'},
        title='Effect of Unemployment Rate on Vehicle Type and Sales'))
        return [
            html.Div(className='chart-item', children=[html.Div(children=R_chart1),html.Div(children=R_chart2)],style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=R_chart3),html.Div(children=R_chart4)],style={'display': 'flex'}),
            ]

    elif(input_year and selected_statistics == 'Yearly Statistics'):
        yearly_data = data[data['Year'] == input_year]
        yas= data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(yas,
                x='Year',
                y='Automobile_Sales',
                title="Automobile sales fluctuate over Recession Period"
             )) 
        mas=data.groupby('Month')["Automobile_Sales"].count().reset_index()
        Y_chart2 = dcc.Graph(figure=px.line(mas,
            x='Month',
            y='Automobile_Sales',
            title='Total Monthly Automobile Sales'))    
        avr_vdata=yearly_data.groupby("Year")["Automobile_Sales"].mean().reset_index()

        Y_chart3 = dcc.Graph(figure=px.plot(avr_vdata, 
                             x="Year",
                             y="Automobile_Sales"               
        ),title='Average Vehicles Sold by Vehicle Type in the year {}'.format(input_year))

        exp_data=yearly_data.groupby('Vehicle_Type')["Advertising_Expenditure"].count().reset_index()
        Y_chart4 = dcc.Graph(
            figure=px.pie(exp_data, 
            values='Advertising_Expenditure',
            names='Vehicle_Type',
            title='Total Advertisment Expenditure for Each Vehicle'))

    return [
                html.Div(className='chart-item', children=[html.Div(children=Y_chart1),html.Div(children=Y_chart2)],style={'display':'flex'}),
                html.Div(className='chart-item', children=[html.Div(children=Y_chart3),html.Div(children=Y_chart4)],style={'display': 'flex'})
                ]
    
    
        


In [8]:
#TASK 2.4: Creating Callbacks
# Define the callback function to update the input container based on the selected statistics
if __name__ == '__main__':
    app.run_server(debug=True)


---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[7], line 70, in update_output_container(
    selected_statistics='Yearly Statistics',
    input_year=1980
)
     64 Y_chart2 = dcc.Graph(figure=px.line(mas,
     65     x='Month',
     66     y='Automobile_Sales',
     67     title='Total Monthly Automobile Sales'))    
     68 avr_vdata=yearly_data.groupby("Year")["Automobile_Sales"].mean().reset_index()
---> 70 Y_chart3 = dcc.Graph(figure=px.plot(avr_vdata, 
        dcc.Graph = <class 'dash.dcc.Graph.Graph'>
        avr_vdata =    Year  Automobile_Sales
0  1980            678.95
        dcc = <module 'dash.dcc' from '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/dash/dcc/__init__.py'>
        px = <module 'plotly.express' from '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/plotly/express/__init__.py'>
        

: 